In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN
from sklearn.decomposition import PCA

In [ ]:
!pip install awscli

In [ ]:
def shell(command):
  return JSON([getoutput(command)])
  output.register_callback('shell',shell)
  !pip install boto3

In [ ]:
import boto3
import s3fs
import awscli
import pandas as pd

In [ ]:
s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

In [ ]:
client = boto3.client('s3')
path = 's3://team3-final-bucket/Resources_clean/objects_clean.csv'

In [ ]:
objects_clean = pd.read_csv(path, index_col=[0])
objects_clean.head()

In [ ]:
# Defining the target and setting lables to 0 and 1
# As we are going to predict start-up failure the "Status" column should be defined as target
# The status in this data set contains a few values: aquired, alpha, beta, closed, development, ipo, live, operating, private
# Closed status will have definition of failure, all pther statuses - non-failure
objects_clean_copy = objects_clean[['entity_type','category_code','status','funding_total_usd']].copy()

In [ ]:
objects_clean_copy.dtypes

In [ ]:
objects_clean_copy['status'].unique()

In [ ]:
objects_clean_copy['status'].replace(to_replace=['closed'], value='0', inplace=True)
objects_clean_copy['status'].replace(to_replace=['operating', 'acquired', 'ipo', 'live', 'beta',
       'private', 'alpha', 'development'], value='1', inplace=True)

In [ ]:
objects_clean_copy.head(20)

In [ ]:
objects_clean_copy['status'] = pd.to_numeric(objects_clean_copy['status'])
objects_clean_copy.dtypes

In [ ]:
# Splitting the table to the features and the target
X = objects_clean_copy.drop(['status','funding_total_usd'], axis=1)
y = objects_clean_copy['status'].values

In [ ]:
X.head()

In [ ]:
y

In [ ]:
# Encode the categorical features
data_encoded_df = pd.get_dummies(X, columns=['entity_type', 'category_code'])
data_encoded_df

In [ ]:
# Create encoded X df with the funding column added
X = data_encoded_df

In [ ]:
# Splitting the data to train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Scale the features 
data_scaler = StandardScaler()
X_scaled = data_scaler.fit_transform(X)

In [ ]:
# Count target train data
Counter(y_train)

In [ ]:
# Resample
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
smoteen_regress = LogisticRegression(solver='lbfgs', random_state=1)
smoteen_regress.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
smoteen_acc = smoteen_regress.predict(X_test)
balanced_accuracy_score(y_test, smoteen_acc)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, smoteen_acc)

In [ ]:
# Print the imbalanced classification report
print("Combination Imbalanced Classification Report" + "\n" + "\n" + classification_report_imbalanced(y_test, smoteen_acc))